In [2]:
import pandas as pd

train_data = pd.read_csv('mnist_train.csv')
test_data = pd.read_csv('mnist_test.csv')

train_data.shape, test_data.shape

((60000, 785), (10000, 785))

In [3]:
X_train = train_data.drop(['label'], axis=1)
y_train = train_data['label']
X_test = test_data.drop(['label'], axis=1)
y_test = test_data['label']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader

class FC_NN(nn.Module):
    def __init__(self):
        super(FC_NN, self).__init__()
        self.input_layer = nn.Linear(784, 128) # input_node, output_node
        self.hidden_layer = nn.Linear(128, 10) # input_node, output_node
        
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        output = self.hidden_layer(x)
        
        return output

torch_model = FC_NN()
loss_fn = nn.CrossEntropyLoss() # softmax 포함
optimizer = torch.optim.Adam(torch_model.parameters(), lr=0.01)

torch_x_train = torch.FloatTensor(X_train.values)
torch_y_train = torch.FloatTensor(y_train.values).long()
torch_x_test = torch.FloatTensor(X_test.values)
torch_y_test = torch.FloatTensor(y_test.values).long()

torch_train = TensorDataset(torch_x_train, torch_y_train)
torch_test = TensorDataset(torch_x_test, torch_y_test)

torch_train_loader = DataLoader(dataset=torch_train, batch_size = 64, shuffle=True)
torch_test_loader = DataLoader(dataset=torch_test, batch_size = 64, shuffle=False)

In [5]:
# torch_model.train()
# n_epochs = 20
# i = 1

# for epoch in range(n_epochs):
#     avg_loss = 0
#     total_batch = len(torch_train_loader)
    
#     for data, targets in torch_train_loader:
#         optimizer.zero_grad()
        
#         x = data.view(-1, 784)
#         prediction = torch_model(x)
#         loss = loss_fn(prediction, targets)
#         loss.backward()
#         optimizer.step()
#         avg_loss += loss / total_batch
    
#     print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")
# print("end")

In [6]:
# torch_model.eval()

# with torch.no_grad():
#     correct = 0
#     for data, targets in torch_train_loader:
#         x = data.view(-1, 784)
#         prediction = torch_model(x)
#         output, predicted = torch.max(prediction, 1)
#         correct += predicted.eq(targets).sum()
#     data_num = len(torch_train_loader.dataset)
#     print(f"accuracy: {correct / data_num : .4f}")
    
#     correct = 0
#     for data, targets in torch_test_loader:
#         x = data.view(-1, 784)
#         prediction = torch_model(x)
#         output, predicted = torch.max(prediction, 1)
#         correct += predicted.eq(targets).sum()
#     data_num = len(torch_test_loader.dataset)
#     print(f"val_accuracy: {correct / data_num : .4f}")

In [7]:
# model = Sequential()
# model.add(Dense(128, input_shape=(784,), activation='relu'))
# model.add(Dense(10, activation='softmax'))
# model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(0.01), metrics=['accuracy'])
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers

def Torch_version(batch_size, epoch):
    torch_model = FC_NN()
    loss_fn = nn.CrossEntropyLoss() # softmax 포함
    optimizer = torch.optim.Adam(torch_model.parameters(), lr=0.01)
    torch_train_loader = DataLoader(dataset=torch_train, batch_size = batch_size)
    torch_test_loader = DataLoader(dataset=torch_test, batch_size = batch_size)
    
    torch_model.train()
    n_epochs = epoch
    i = 1

    for epoch in range(n_epochs):
        avg_loss = 0
        total_batch = len(torch_train_loader)

        for data, targets in torch_train_loader:
            optimizer.zero_grad()

            x = data.view(-1, 784)
            prediction = torch_model(x)
            loss = loss_fn(prediction, targets)
            loss.backward()
            optimizer.step()
            avg_loss += loss / total_batch
            
    torch_model.eval()

    with torch.no_grad():
        correct_train = 0; correct_test = 0
        for data, targets in torch_train_loader:
            x = data.view(-1, 784)
            prediction = torch_model(x)
            output, predicted = torch.max(prediction, 1)
            correct_train += predicted.eq(targets).sum()
        data_num_train = len(torch_train_loader.dataset)

        correct = 0
        for data, targets in torch_test_loader:
            x = data.view(-1, 784)
            prediction = torch_model(x)
            output, predicted = torch.max(prediction, 1)
            correct_test += predicted.eq(targets).sum()
        data_num_test = len(torch_test_loader.dataset)
        
    return avg_loss, correct_train / data_num_train, correct_test / data_num_test

def Torch_version_only_train(batch_size, epoch):
    torch_model = FC_NN()
    loss_fn = nn.CrossEntropyLoss() # softmax 포함
    optimizer = torch.optim.Adam(torch_model.parameters(), lr=0.01)
    torch_model.train()

    for epoch in range(epoch):
        avg_loss = 0
        total_batch = len(torch_train_loader)

        for data, targets in torch_train_loader:
            optimizer.zero_grad()

            x = data.view(-1, 784)
            prediction = torch_model(x)
            loss = loss_fn(prediction, targets)
            loss.backward()
            optimizer.step()
        
    return avg_loss, 0, 0

def Tensor_version(batch_size, epoch):
    model = Sequential()
    model.add(Dense(128, input_shape=(784,), activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(0.01), metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch, batch_size=batch_size, verbose=False)
    
    return history.history['loss'][-1], history.history['accuracy'][-1], history.history['val_accuracy'][-1]

In [13]:
import time

for i in range(1,6):
    start = time.time()
    loss, acc, val_acc = Tensor_version(64, 20)
    end = time.time()
    print(f"| Tenflow {i} | time: {end-start:.4f} | loss: {loss:.4f} | acc: {acc:.4f} | val_acc: {val_acc:.4f} |")
#     start = time.time()
#     loss, acc, val_acc = Torch_version(64, 20)
#     end = time.time()
#     print(f"| PyTorch {i} | time: {end-start:.4f} | loss: {loss:.4f} | acc: {acc:.4f} | val_acc: {val_acc:.4f} |")
#     start = time.time()
#     loss, acc, val_acc = Torch_version_only_train(64, 20)
#     end = time.time()
#     print(f"| Torchot {i} | time: {end-start:.4f} | loss: {loss:.4f} | acc: {acc:.4f} | val_acc: {val_acc:.4f} |")

| Tenflow 1 | time: 19.8539 | loss: 0.0608 | acc: 0.9877 | val_acc: 0.9679 |
| Tenflow 2 | time: 19.7092 | loss: 0.0540 | acc: 0.9885 | val_acc: 0.9720 |
| Tenflow 3 | time: 19.7319 | loss: 0.0522 | acc: 0.9889 | val_acc: 0.9688 |
| Tenflow 4 | time: 19.7392 | loss: 0.0538 | acc: 0.9887 | val_acc: 0.9664 |
| Tenflow 5 | time: 19.9213 | loss: 0.0523 | acc: 0.9890 | val_acc: 0.9721 |
